## Example: Converting a Segmentation Stack + geff for Use with TrackGardener

This notebook demonstrates how to convert a segmentation stack and a [GEFF](https://github.com/live-image-tracking-tools/geff) graph into a TrackGardener database using the configuration file Fluo-N2DL-HeLa-01_config_geff.yaml.

In this workflow, the segmentation stack provides the precise segmentation masks, while the GEFF file supplies only the graph structure. The GEFF file is expected to specify both `t` (timepoint), `track_id` (track ID) and `segm_id` (segmentation ID) for each cell entry. The notebook verifies that for each timepoint t, there is a corresponding object in the segmentation array (labeled stack) with the matching segm_id. For further details on additional checks, please refer to the documentation.

All signals are computed according to the specifications in the configuration file. Please note that this process may take a considerable amount of time for large datasets.

You can reuse the configuration file shown here with the TrackGardener plugin. Be sure to update the file paths in the configuration to point to your imaging dataset and TrackGardener database. It is recommended to use absolute paths (instead of the relative paths used in this example) to ensure correct file location. If you prefer to use relative paths, they should be specified relative to the location of the configuration file.

In [1]:
import dask.array as da
import yaml

from track_gardener.converters import (
    segm_and_geff_to_TG,
    validate_geff_seg_ids,
)

In [2]:
geff_group_path = "Fluo-N2DL-HeLa-01_sample.geff"
segmentation_path = "Fluo-N2DL-HeLa-01_segm_random.zarr/labels"
config_path = 'Fluo-N2DL-HeLa-01_config_geff.yaml'

#### Validate a match betwen GEFF and the segmentation stack

In [3]:
p, details = validate_geff_seg_ids(
    geff_group_path = geff_group_path,
    segmentation_path = segmentation_path,
    seg_id_field = "segm_id",
    check_xy_position = True,
    check_extra_segmentations = True)

print(f'Test passed: {p}, details: {details}.')

2025-08-12 21:24:23.221 | INFO     | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:31 - Opening GEFF group from: Fluo-N2DL-HeLa-01_sample.geff
2025-08-12 21:24:23.279 | INFO     | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:52 - Beginning node validation...
2025-08-12 21:24:24.535 | INFO     | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:93 - Checking for extra segmentations not referenced in GEFF...
2025-08-12 21:24:24.765 | SUCCESS  | track_gardener.converters.validator_seg_id:validate_geff_seg_ids:112 - All validations passed successfully.


Test passed: True, details: {'node_issues': [], 'extra_segments': {}}.


#### Convert to the TrackGardener database

In [4]:
# read in configuration file
with open(config_path) as file:
    config = yaml.safe_load(file)

# get segmentation array
segm_array = da.from_zarr(segmentation_path)

In [5]:
segm_and_geff_to_TG(segm_array, geff_group_path, config)

### Validate the resulting database

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from track_gardener.db.db_validate import run_tracking_db_checks

In [7]:
db_path = config['database']['path']
engine = create_engine(f'sqlite:///{db_path}')
Session = sessionmaker(bind=engine)

In [8]:
# empty list indicates no errors
with Session() as session:
    errors = run_tracking_db_checks(session)

errors

[]